# Step-by-step walkthrough of Etch contract development and deployment

### Installation (for first-time use)

Install Anaconda (if not done already): [Download Link](<https://docs.anaconda.com/anaconda/install/>)

Setup Python env "etch" for Etch contract development. It is currently tested for Python3.6.

```
conda update conda -y
conda create -n etchenv python=3.6 -y
conda activate etchenv
```

Pip install Ledger Python API. The latest version is currently alpha v0.10.x. [Resource](https://docs.fetch.ai/getting-started/python-api-install/)

`pip install -U fetchai-ledger-api`

Alternatively you can install directly from source:

```
git clone https://github.com/fetchai/ledger-api-py.git
cd ledger-api-py
pip uninstall fetchai-ledger-api
```

Add etchenv as an iPython kernel and restart Jupyter Notebook. That way you can see "etchenv" as a kernel option.

`python -m ipykernel install --user --name etchenv --display-name "etch"`

Install the Fetch ledger at your designated directory. See see Fetch [docs](https://docs.fetch.ai/) "Getting Started" section for instructions to do so. Once built (including Constellation), move to the following directory:

`cd ledger/build/apps/constellation`

Run the Fetch ledger on your local testnode with the following command:

`./constellation -port 8100 -block-interval 3000 -standalone`

The output will show the following in console:

```
F E ╱     Constellation v0.4.1-rc1
   T C     Copyright 2018-2019 (c) Fetch AI Ltd.
     H

[ 2019-06-03 16:55:20.215, # 1 INFO  :                                main ] Configuration:

port......................: 8100
network mode..............: Standalone
num executors.............: 1
num lanes.................: 1
num slices................: 500
bootstrap.................: 0
discoverable..............: 0
host name.................:
external address..........: 127.0.0.1
db-prefix.................: node_storage
interface.................: 127.0.0.1
mining....................: Yes
tx processor threads......: 12
shard verification threads: 12
block interval............: 3000ms
max peers.................: 3
peers update cycle........: 0ms
peers.....................:
manifest.......:
 - HTTP/0: tcp://127.0.0.1:8100 (8100)
 - CORE/0: tcp://127.0.0.1:8101 (8101)
 - Lane/0: tcp://127.0.0.1:8110 (8110)
```

### Etch Development Walkthrough

#### Step 1) Setup Python API and initialize contract owner entity

Print the Ledger Python API version. Make sure that it is the latest. 

In [1]:
from fetchai.ledger import __version__ as version_string
print(version_string)

0.10.0-a6


Import the following to access Ledger Python API

In [2]:
from fetchai.ledger.api import LedgerApi
from fetchai.ledger.contract import Contract
from fetchai.ledger.crypto import Entity, Address

Set up Python API, using your local config: 

In [3]:
# Constellation config
HOST = '127.0.0.1'
PORT = 8100

Create an API instance

In [4]:
api = LedgerApi(HOST, PORT)

Create an Entity for owner of the Etch contract, where contains owner's public/private keypair as well as its wallet address. In the pet-shop example, the owner can be the shelter iteself.

In [5]:
# Create keypair for the contract owner
owner = Entity()
owner_addr = Address(owner)

In [6]:
print(owner_addr)

2Bv7pM3tht2wURL2dumNM7ess497NfX7CTtqHEcJGherfSYvMn


(Optional) Save the contract owner's private key to disk with a password

In [7]:
with open('owner_private.key', 'w') as private_key_file:
    owner.dump(private_key_file, password="pass")

#### Step 2) Deploy the pet-shop contract

Read-in script from adoption.etch

In [21]:
source = "pet-shop/adoption.etch"

with open(source, 'r') as file:
    contract_text = file.read()

Print adoption contract text:

In [22]:
print(contract_text)

//------------------------------------------------------------------------------
//
//   Copyright 2019 Fetch.AI Limited
//
//   Licensed under the Apache License, Version 2.0 (the "License");
//   you may not use this file except in compliance with the License.
//   You may obtain a copy of the License at
//
//       http://www.apache.org/licenses/LICENSE-2.0
//
//   Unless required by applicable law or agreed to in writing, software
//   distributed under the License is distributed on an "AS IS" BASIS,
//   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
//   See the License for the specific language governing permissions and
//   limitations under the License.
//
//------------------------------------------------------------------------------

// This contract require release v0.10.1 alpha or above
// Note that this is experimental functionality

persistent adopters : Array<Address>;


@init
function constructor()

    use adopters;
    adopters.set(Array<Ad

***TO-DO***: writeup about the contract desc

Create contract object from contract text:

In [23]:
contract = Contract(contract_text, owner)

Top-up 20,000 FET tokens to owner entity in order to pay for ledger tx fees. Note: this is your local testnet so you can print new tokens at-will.

In [25]:
api.sync(api.tokens.wealth(owner, 20000))

Print the owner's balance to confirm the 20,000 has been received

In [26]:
print(api.tokens.balance(address=owner_addr))

30000


Deploy the contract on-chain, paying 10,000 in transaction fees (an arbitrary amount).

In [27]:
api.sync(api.contracts.create(owner, contract, 10000))

RuntimeError: Some transactions have failed: 85a2bb6af13b922d5a8d75a8006a7cf2aa3c0a0bb83ad9a689d913d15fd2bc2e:Contract Execution Failure

Alternatively Execute `python deploy.py pet` to deploy pet-shop contract.

#### Step 3) Interact with the pet-shop contract

Create keypair for new entity that interacts with contract. In this case, this will be adopter looking to adopt a pet from the shelter.

In [ ]:
adopter1 = Entity()
adopter_addr1 = Address(adopter1)

In [ ]:
print(adopter_addr1)

Top-up the adopter entity as well as the user also needs FET tokens to pyay for tx fees when interacting the the contract.

In [ ]:
api.sync(api.tokens.wealth(adopter1, 10000))

First, query *getAdopters* function to see which pets are adopted.

In [ ]:
# Printing message
result = contract.query(api, 'getAdopters')

In [ ]:
print(result)

Next, pick a pet to adopt. Let's say pet on index 10.

In [ ]:
# Initialize tx fee
tok_transfer_amount = 200
fet_tx_fee = 100

In [ ]:
# Execute smart contract5
pet_id = 10
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [adopter1], pet_id))

Query *getAdopters* to confirm that pet on index 10 has been adopted by adopter1, with adopter1's address listed.

In [ ]:
# Printing message
print(contract.query(api, 'getAdopters'))

Let's create anther adopter.

In [ ]:
adopter2 = Entity()
adopter_addr2 = Address(adopter2)

In [ ]:
print(adopter_addr2)

In [ ]:
# Top-up the adopter
api.sync(api.tokens.wealth(adopter2, 10000))

Adopter2 will first try to pet on index 10. However, that execution should fail as the pet is already have taken by Adopter1.

In [ ]:
pet_id = 10
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [adopter1], pet_id))

Thus, Adopter2 will settle with the pet on index 5 instead:

In [ ]:
pet_id = 5
result = api.sync(contract.action(api, 'adopt', fet_tx_fee, [adopter1], pet_id))

Let's take a look at *getAdopters* one last time:

In [ ]:
# Printing message
print(contract.query(api, 'getAdopters'))

#### Step 4) Deploy the simple-open-auction contract

Now let's try to deploy a more challenging auction contract.

In [28]:
source = "simple-open-auction/auction.etch"

with open(source, 'r') as file:
    contract_text = file.read()

Print auction contract text:

In [29]:
print(contract_text)

 //------------------------------------------------------------------------------
//
//   Copyright 2019 Fetch.AI Limited
//
//   Licensed under the Apache License, Version 2.0 (the "License");
//   you may not use this file except in compliance with the License.
//   You may obtain a copy of the License at
//
//       http://www.apache.org/licenses/LICENSE-2.0
//
//   Unless required by applicable law or agreed to in writing, software
//   distributed under the License is distributed on an "AS IS" BASIS,
//   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
//   See the License for the specific language governing permissions and
//   limitations under the License.
//
//------------------------------------------------------------------------------

// This contract require release v0.9.1 or above
// Note that this is experimental functionality

persistent beneficiary : Address;
persistent auctionEndTime : UInt64;
persistent highestBidder : Address;
persistent hi

***TO-DO***: writeup about the contract desc

Create contract object from contract text:

In [30]:
contract = Contract(contract_text, owner)

Top-up 20,000 FET tokens to owner entity in order to pay for ledger tx fees. Note: this is your local testnet so you can print new tokens at-will.

In [31]:
api.sync(api.tokens.wealth(owner, 20000))

Print the owner's balance to confirm the 20,000 has been received

In [32]:
print(api.tokens.balance(address=owner_addr))

40000


Deploy the contract on-chain, paying 10,000 in transaction fees (an arbitrary amount).

In [33]:
api.sync(api.contracts.create(owner, contract, 10000))

RuntimeError: Some transactions have failed: aed347475029ca4336523767f45f66d1dc1222f178ece397e1feec89b093bc8f:Contract Execution Failure

Check the current block number:

Alternatively Execute `python deploy.py pet` to deploy pet-shop contract.

#### Step 5) Interact with the simple-open-auction contract

Create keypair for new entity that interacts with contract. In this case, this will be adopter looking to adopt a pet from the shelter.

In [ ]:
bidder1 = Entity()
bidder_addr1 = Address(adopter1)

print(bidder1)

Top-up the adopter entity as well as the user also needs FET tokens to pyay for tx fees when interacting the the contract.

In [ ]:
api.sync(api.tokens.wealth(bidder1, 10000))

First, query *getAdopters* function to see which pets are adopted.

In [ ]:
# Printing message
print(contract.query(api, 'getAdopters'))

Next, pick a pet to adopt. Let's say pet on index 10.

In [ ]:
# Initialize tx fee
tok_transfer_amount = 200
fet_tx_fee = 100

make a bid amount:

In [ ]:
bid_amount = 2000 
api.sync(contract.action(api, 'bid', fet_tx_fee, bid_amount)